# Lab 4: Recurrent Neural Networks

Veľmi často potrebujete natrénovať model na údaje, ktoré reprezentujú časovú postupnosť, t.j. hodnota prvku závisí od hodnoty predošlých prvkov. Na takéto údaje sú najvhodnejšie rekurentné neurónové siete (recurrent neural network). Na dnešnom cvičení použijete rekurentnú neurónku na generovanie textu.

Pred tým sa ale oboznámite s [problematikou časových postupností](http://introtodeeplearning.com/materials/2019_6S191_L2.pdf)

Kostru dnešného riešenia nájdete [tu](https://github.com/ianmagyar/dl-course/blob/master/labs/sources/lab4.py). Stiahnite si aj [dataset s textmi piesní kapely Queen](https://github.com/ianmagyar/dl-course/blob/master/labs/sources/dataset_queen.txt).

## 1. Načítanie datasetu

V prvom kroku načítame dataset ako postupnosť znakov. Na to použijeme funkciu open z jazyka Python. Následne potrebujete priradiť hodnotu nasledujúcim premenným:

* chars - zoznam jedinečných znakov v datasete
* VOCAB_SIZE - počet jedinečných znakov v datasete
* ix_to_char - slovník, ktorý prepíše index znaku na znak
* char_to_ix - slovník, ktorý nahradí znak jeho indexom

In [ ]:
data = open('dataset_queen.txt', 'r').read()
# TODO: get a list of all unique characters
chars = 
# TODO: get the number of unique letters
VOCAB_SIZE = 

# TODO: create dictionaries that map index to letters and vice versa
ix_to_char = 
char_to_ix = 


## 2. Príprava vstupných a výstupných hodnôt

Keďže našim cieľom je generovať úplne nové texty, nepoužijeme trénovaciu množinu a testovaciu množinu, ale hodnoty aj tak potrebujeme predspracovať, keďže texty sú zatiaľ reprezentované ako znaky resp. indexy, ale naša neurónka bude očakávať na vstupe aj na výstupe vektory. Je to potrebné kvôli tomu aby sme problém reprezentovali ako klasifikáciu namiesto regresie.

Vstupom pre neurónovú sieť bude postupnosť znakov a výstupom bude postupnosť znakov posunutá o jednu pozíciu. Ak teda chceme natrénovať neurónku pomocou reťazca "hello", tak vstupom bude "hell" a výstup bude "ello".

Najprv teda potrebujeme zadefinovať dĺžku jedného reťazca a následne aj počet reťazcov v datasete.

In [ ]:
# TODO: define the length of one sequence and calculate the number of sequences
SEQ_LENGTH = 
SEQ_NUMBER = 

Ďalšou úlohou je pripraviť vstupy a výstupy tak, aby sme vedeli začať trénovať model. Každý vstup bude reprezentovaný ako pole `SEQ_LENGTH - 1` vektorov. V kóde máte príklad na prípravu vstupu, na základe čoho viete napísať podobný kód na prípravu výstupu. Nezabudnite že výstup má byť reťazec posunutý o jeden znak.

In [ ]:
# create vector representation of input and output
X = np.zeros((SEQ_NUMBER, SEQ_LENGTH, VOCAB_SIZE))
y = np.zeros((SEQ_NUMBER, SEQ_LENGTH, VOCAB_SIZE))
for i in range(0, SEQ_NUMBER):
    X_sequence = data[i * SEQ_LENGTH:(i + 1) * SEQ_LENGTH]
    X_sequence_ix = [char_to_ix[value] for value in X_sequence]
    input_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        input_sequence[j][X_sequence_ix[j]] = 1.
    X[i] = input_sequence

    # TODO: prepare output vectors
    

## 3. Definícia modelu

V skripte máte zadefinovaný už rekurentný model. Analyzujte model a zistite úlohu jednotlivých vrstiev.

In [ ]:
# define number of neurons in hidden layers
HIDDEN_DIM = 128

# define model
model = Sequential()
model.add(LSTM(HIDDEN_DIM, input_shape=(None, VOCAB_SIZE), return_sequences=True))
model.add(LSTM(HIDDEN_DIM, return_sequences=True))
model.add(LSTM(HIDDEN_DIM, return_sequences=True))
model.add(TimeDistributed(Dense(VOCAB_SIZE)))
model.add(Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")


## 4. Trénovanie a testovanie modelu

Teraz môžeme začať už testovať náš model. Pre porovnanie vygenerujeme postupnosť 100 znakov ešte pred trénovaním modelu. Na to použijeme funkciu `generate_text`.

In [ ]:
def generate_text(model, length):
    ix = [np.random.randint(VOCAB_SIZE)]
    y_char = [ix_to_char[ix[-1]]]
    X = np.zeros((1, length, VOCAB_SIZE))
    for i in range(length):
        X[0, i, :][ix[-1]] = 1
        print(ix_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i + 1, :])[0], 1)
        y_char.append(ix_to_char[ix[-1]])
    return ('').join(y_char)

In [ ]:
GENERATE_LENGTH = 100
# random generation
generate_text(model, GENERATE_LENGTH)

Pre ukážku trénovanie uskutočníme v nekončenom cyklu (ak kód spúštate cez Jupyter, opravte to na `for` cyklus). Po každom kole trénovanie vygenerujeme nový reťazec a vypíšeme ho pomocou funkcie `generate_text`.

In [ ]:
# train and test model
epoch_no = 0
while True:
    print('\n')
    model.fit(X, y, batch_size=48, verbose=2, epochs=1)
    epoch_no += 1
    generate_text(model, GENERATE_LENGTH)

Ako vidíte, síce nedostaneme zrozumiteľný text na začiatku, ale hneď je jasné, že model sa učí úspešne a snaží sa napodobňovať texty z datasetu. Ak ste zvedaví, aký výkon majú rekurentné neurónky po dlhšom trénovaní, prečítajte si [Harryho Pottera od umelej inteligencie](https://medium.com/deep-writing/harry-potter-written-by-artificial-intelligence-8a9431803da6).

Pomocou rekurentných sietí je možné generovať aj hudbu, ako to robí aj [AIVA](https://www.aiva.ai). Pre ukážku si pozrite video z [TED](https://www.youtube.com/watch?v=wYb3Wimn01s).